In [1]:
# Importar librerias
import pandas as pd
import numpy as np

import zipfile
from datetime import date
from sklearn import metrics
from sklearn.metrics import accuracy_score


from hyperopt import hp, fmin, tpe, hp, STATUS_OK, Trials

from numpy.testing import assert_allclose
from keras.models import Sequential, load_model
from keras.layers import LSTM, Dropout, Dense
from keras.callbacks import ModelCheckpoint
from keras.optimizers import Adadelta

import os
from pathlib import Path


Using TensorFlow backend.


In [2]:
# Seed value
# Apparently you may use different seed values at each stage
seed_value= 0

# 1. Set `PYTHONHASHSEED` environment variable at a fixed value
import os
os.environ['PYTHONHASHSEED']=str(seed_value)

# 2. Set `python` built-in pseudo-random generator at a fixed value
import random
random.seed(seed_value)

# 3. Set `numpy` pseudo-random generator at a fixed value
import numpy as np
np.random.seed(seed_value)

# 4. Set the `tensorflow` pseudo-random generator at a fixed value
import tensorflow as tf
tf.random.set_seed(seed_value)

In [3]:
exp_name = 'Embed_promedio'
folder = 'Resultados/' + exp_name
my_file = Path(folder)
if os.path.exists(my_file):
    print('already exists')
else:
    os.makedirs(folder)
    


already exists


In [4]:
ch_folder = folder + '/Checkpoints'
my_file = Path(ch_folder)
if os.path.exists(my_file):
    print('already exists')
else:
    os.makedirs(ch_folder)

already exists


### Read datasets

In [5]:
#Importar los datasets
url_embeddings_average_individual = zipfile.ZipFile('../Data/embeddings_average_individual.zip')
url_embeddings_sum_individual = zipfile.ZipFile('../Data/embeddings_sum_individual.zip')

embeddings_average_individual = pd.read_csv(url_embeddings_average_individual.open('embeddings_average_individual.csv'), index_col = 0)
embeddings_sum_individual =pd.read_csv(url_embeddings_sum_individual.open('embeddings_sum_individual.csv'), index_col = 0)

embeddings_average_individual['Date'] =  pd.to_datetime(embeddings_average_individual['Date'], format='%Y-%m-%d')
# embeddings_average_individual.sort_values('Date', inplace=True)
# embeddings_average_individual.reset_index(drop = True , inplace=True)

embeddings_average_individual.tail(2)

,0,1,2,3,4,5,6,7,8,9,...,293,294,295,296,297,298,299,Label,Date,Top
49716,-0.082072,0.099915,-0.015503,0.115560,-0.072611,0.070435,0.042613,-0.041026,0.012126,-0.020508,...,-0.062215,-0.078471,0.046008,0.005717,-0.071452,0.122559,0.07622,0,2008-08-08,24
49717,-0.064514,0.013916,-0.028976,0.058716,-0.078369,-0.057312,-0.077515,-0.234467,0.050751,0.020508,...,-0.010498,0.081284,0.040283,-0.108978,0.033783,0.028870,0.03418,0,2008-08-08,25


### Embedding Promedio

In [6]:
# Selecciono la fecha para la cual hago el corte de train y test
training_end = pd.to_datetime("2013-12-31")
num_training = len(embeddings_average_individual[(embeddings_average_individual["Date"]) <= training_end])

In [7]:
# Selecciono el archivo con el que se corre el modelo
data = embeddings_average_individual[embeddings_average_individual['Date']<='2014-12-31']

In [8]:
# Se separa en train y test
x_train = data.drop(["Top","Label", "Date"], axis=1)[:num_training]
x_test = data.drop(["Top",'Label', 'Date'], axis=1)[num_training:]
y_train = data["Label"].values[:num_training]
y_test = data["Label"].values[num_training:]


x_train_array = x_train.to_numpy()
reshape_x_train = x_train_array.reshape(len(x_train), 1, 300)
reshape_x_train.shape

x_test_array = x_test.to_numpy()
reshape_x_test = x_test_array.reshape(len(x_test), 1, 300)
reshape_x_test.shape

(6300, 1, 300)

### Definir espacio de busqueda

In [9]:
space = {
    'units1': hp.choice('units1', [64, 128, 256, 512]),
    'units2': hp.choice('units2', [64, 128, 256, 512]),
                 
    'dropout1': hp.choice('dropout1', [0.2,0.3,0.5]),
    'dropout2': hp.choice('dropout2', [0.2,0.3,0.5]),
    
    'batch_size' : hp.choice('batch_size', [28,64,128]),
    'nb_epochs' : hp.choice('nb_epochs', [50,100]),

    'optimizer':  hp.choice('optimizer', [ 'adam','adadelta']),   
    'activation': 'relu'    
    }

### Definir busqueda bayesiana

In [10]:
#Objective function that hyperopt will minimize
from timeit import default_timer as timer

def objective(params):
    
    import ml_metrics

    
    start = timer()
    print ('Params testing: ', params)
    print ('\n ')
    
    model = Sequential()
    model.add(LSTM(params['units1'], input_shape=(1,300), return_sequences=True))
    model.add(Dropout(params['dropout1']))
    model.add(LSTM(params['units2'], return_sequences=False))
    model.add(Dense(1, activation='sigmoid'))
    #model.compile(loss='mean_squared_error', optimizer='adam')
    # compile the model
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


    
    #includes the call back object
    model.fit(reshape_x_train, y_train, epochs=params['nb_epochs'], batch_size=params['batch_size'],
              verbose = 0, validation_data=(reshape_x_test, y_test))
     
    #predict the test set 
    score, acc = model.evaluate(reshape_x_test, y_test, verbose=0)
    
    run_time = timer() - start
    
    # Write to the csv file ('a' means append)
    of_connection = open(out_file, 'a')
    writer = csv.writer(of_connection)
    writer.writerow([-acc, params, score, run_time])
    of_connection.close()
    
    
    print('Test accuracy:', acc)
 
    return {'loss': -acc, 'status': STATUS_OK, 'train_time': run_time,}



### Almacenar resultados de cada iteración

In [11]:
from hyperopt import tpe

tpe_algorithm = tpe.suggest

In [12]:
import csv

from hyperopt import Trials

bayes_trials = Trials()

# File to save first results
out_file = folder + '/gbm_results.csv'
of_connection = open(out_file, 'w')

writer = csv.writer(of_connection)

# Write the headers to the file
writer.writerow(['loss', 'params', 'score','time'])
of_connection.close()

### Lanzar optimización

In [13]:
trials = Trials()


# Run optimization
best = fmin(fn = objective, space = space, algo = tpe.suggest, 
            max_evals = 20, trials = bayes_trials,
            verbose = 1, rstate= np.random.RandomState(50))


Params testing:                                                                                                        
{'activation': 'relu', 'batch_size': 128, 'dropout1': 0.3, 'dropout2': 0.3, 'nb_epochs': 50, 'optimizer': 'adadelta', 'units1': 256, 'units2': 512}
Test accuracy:                                                                                                         
0.49492064118385315                                                                                                    
Params testing:                                                                                                        
{'activation': 'relu', 'batch_size': 128, 'dropout1': 0.3, 'dropout2': 0.3, 'nb_epochs': 50, 'optimizer': 'adadelta', 'units1': 64, 'units2': 512}
Test accuracy:                                                                                                         
0.4938095211982727                                                                                       

Params testing:                                                                                                        
{'activation': 'relu', 'batch_size': 64, 'dropout1': 0.5, 'dropout2': 0.2, 'nb_epochs': 100, 'optimizer': 'adam', 'units1': 256, 'units2': 64}
Test accuracy:                                                                                                         
0.5003174543380737                                                                                                     
Params testing:                                                                                                        
{'activation': 'relu', 'batch_size': 64, 'dropout1': 0.3, 'dropout2': 0.5, 'nb_epochs': 100, 'optimizer': 'adadelta', 'units1': 512, 'units2': 128}
Test accuracy:                                                                                                         
0.48253968358039856                                                                                          

### Exportar bayesiana, por si quisiera retomar donde queda

In [14]:
import pickle
pickle.dump(bayes_trials, open(folder + '/trials.pkl', 'wb'))

### Leer mejores parametros

In [15]:
# Sort the trials with lowest loss (highest AUC) first
bayes_trials_results  = sorted(bayes_trials.results, key = lambda x: x['loss'])
bayes_trials_results [:2]

[{'loss': -0.5026984214782715,
  'status': 'ok',
  'train_time': 226.2653377000006},
 {'loss': -0.502063512802124, 'status': 'ok', 'train_time': 1841.8912946}]

In [16]:
results = pd.read_csv(folder + '/gbm_results.csv')

# Sort with best scores on top and reset index for slicing
results.sort_values('loss', ascending = True, inplace = True)
results.reset_index(inplace = True, drop = True)
results.head()

,loss,params,score,time
0,-0.502698,"{'activation': 'relu', 'batch_size': 128, 'dro...",NaN,226.265338
1,-0.502064,"{'activation': 'relu', 'batch_size': 28, 'drop...",NaN,1841.891295
2,-0.500635,"{'activation': 'relu', 'batch_size': 128, 'dro...",NaN,781.529357
3,-0.500317,"{'activation': 'relu', 'batch_size': 64, 'drop...",NaN,1028.309983
4,-0.498730,"{'activation': 'relu', 'batch_size': 28, 'drop...",NaN,388.266052


In [17]:
import ast

# Convert from a string to a dictionary
ast.literal_eval(results.loc[0, 'params'])

{'activation': 'relu',
 'batch_size': 128,
 'dropout1': 0.2,
 'dropout2': 0.3,
 'nb_epochs': 50,
 'optimizer': 'adadelta',
 'units1': 64,
 'units2': 256}

In [18]:
# Extract the ideal number of estimators and hyperparameters
best_bayes_params = ast.literal_eval(results.loc[0, 'params']).copy()
best_bayes_params

{'activation': 'relu',
 'batch_size': 128,
 'dropout1': 0.2,
 'dropout2': 0.3,
 'nb_epochs': 50,
 'optimizer': 'adadelta',
 'units1': 64,
 'units2': 256}

### Definir datasets de testeo

In [19]:
# Selecciono la fecha para la cual hago el corte de train y test
training_end = pd.to_datetime("2014-12-31")
num_training = len(embeddings_average_individual[(embeddings_average_individual["Date"]) <= training_end])

In [20]:
# Selecciono el archivo con el que se corre el modelo
data = embeddings_average_individual

In [21]:
# Se separa en train y test
x_train = data.drop(["Top","Label", "Date"], axis=1)[:num_training]
x_test = data.drop(["Top",'Label', 'Date'], axis=1)[num_training:]
y_train = data["Label"].values[:num_training]
y_test = data["Label"].values[num_training:]


x_train_array = x_train.to_numpy()
reshape_x_train = x_train_array.reshape(len(x_train), 1, 300)
reshape_x_train.shape

x_test_array = x_test.to_numpy()
reshape_x_test = x_test_array.reshape(len(x_test), 1, 300)
reshape_x_test.shape

(9450, 1, 300)

In [22]:
model = Sequential()
model.add(LSTM(best_bayes_params['units1'], input_shape=(1,300), return_sequences=True))
model.add(Dropout(best_bayes_params['dropout1']))
model.add(LSTM(best_bayes_params['units2'], return_sequences=False))
model.add(Dense(1, activation='sigmoid'))
#model.compile(loss='mean_squared_error', optimizer='adam')
# compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


#model.compile(loss='mean_squared_error', optimizer='adam')
# compile the model
model.compile(loss='binary_crossentropy',
          optimizer='adam',
          metrics=['accuracy'])


# define the checkpoint
filepath= ch_folder + "/word2vec-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [23]:
# fit the model
model.fit(reshape_x_train, y_train,
          epochs=best_bayes_params['nb_epochs'], 
          batch_size=best_bayes_params['batch_size'], callbacks=callbacks_list)


model.save(folder + '/keras_model.h5')
model.summary()

Epoch 1/50
40268/40268 [==============================] - ETA: 4:49 - loss: 0.6934 - accuracy: 0.47 - ETA: 1:42 - loss: 0.6934 - accuracy: 0.45 - ETA: 1:04 - loss: 0.6932 - accuracy: 0.48 - ETA: 42s - loss: 0.6928 - accuracy: 0.5068 - ETA: 32s - loss: 0.6928 - accuracy: 0.509 - ETA: 26s - loss: 0.6920 - accuracy: 0.522 - ETA: 22s - loss: 0.6913 - accuracy: 0.529 - ETA: 19s - loss: 0.6903 - accuracy: 0.537 - ETA: 18s - loss: 0.6907 - accuracy: 0.535 - ETA: 16s - loss: 0.6920 - accuracy: 0.529 - ETA: 15s - loss: 0.6915 - accuracy: 0.531 - ETA: 14s - loss: 0.6918 - accuracy: 0.529 - ETA: 13s - loss: 0.6919 - accuracy: 0.529 - ETA: 13s - loss: 0.6916 - accuracy: 0.530 - ETA: 12s - loss: 0.6912 - accuracy: 0.533 - ETA: 11s - loss: 0.6907 - accuracy: 0.537 - ETA: 11s - loss: 0.6911 - accuracy: 0.534 - ETA: 10s - loss: 0.6909 - accuracy: 0.535 - ETA: 10s - loss: 0.6909 - accuracy: 0.536 - ETA: 10s - loss: 0.6905 - accuracy: 0.537 - ETA: 9s - loss: 0.6906 - accuracy: 0.537 - ETA: 9s - loss: 0.

40268/40268 [==============================] - ETA: 6s - loss: 0.6933 - accuracy: 0.52 - ETA: 6s - loss: 0.6908 - accuracy: 0.53 - ETA: 6s - loss: 0.6907 - accuracy: 0.53 - ETA: 6s - loss: 0.6940 - accuracy: 0.51 - ETA: 6s - loss: 0.6929 - accuracy: 0.52 - ETA: 6s - loss: 0.6911 - accuracy: 0.53 - ETA: 6s - loss: 0.6905 - accuracy: 0.53 - ETA: 6s - loss: 0.6908 - accuracy: 0.53 - ETA: 6s - loss: 0.6914 - accuracy: 0.53 - ETA: 5s - loss: 0.6911 - accuracy: 0.53 - ETA: 5s - loss: 0.6905 - accuracy: 0.53 - ETA: 5s - loss: 0.6902 - accuracy: 0.53 - ETA: 5s - loss: 0.6901 - accuracy: 0.53 - ETA: 5s - loss: 0.6906 - accuracy: 0.53 - ETA: 5s - loss: 0.6911 - accuracy: 0.53 - ETA: 5s - loss: 0.6908 - accuracy: 0.53 - ETA: 5s - loss: 0.6907 - accuracy: 0.53 - ETA: 5s - loss: 0.6908 - accuracy: 0.53 - ETA: 5s - loss: 0.6906 - accuracy: 0.53 - ETA: 5s - loss: 0.6908 - accuracy: 0.53 - ETA: 5s - loss: 0.6907 - accuracy: 0.53 - ETA: 5s - loss: 0.6907 - accuracy: 0.53 - ETA: 5s - loss: 0.6907 - accu

40268/40268 [==============================] - ETA: 5s - loss: 0.6888 - accuracy: 0.56 - ETA: 6s - loss: 0.6894 - accuracy: 0.54 - ETA: 6s - loss: 0.6905 - accuracy: 0.53 - ETA: 6s - loss: 0.6901 - accuracy: 0.53 - ETA: 5s - loss: 0.6898 - accuracy: 0.54 - ETA: 5s - loss: 0.6891 - accuracy: 0.54 - ETA: 5s - loss: 0.6890 - accuracy: 0.54 - ETA: 5s - loss: 0.6894 - accuracy: 0.54 - ETA: 5s - loss: 0.6881 - accuracy: 0.54 - ETA: 5s - loss: 0.6879 - accuracy: 0.54 - ETA: 5s - loss: 0.6859 - accuracy: 0.55 - ETA: 5s - loss: 0.6862 - accuracy: 0.55 - ETA: 5s - loss: 0.6865 - accuracy: 0.55 - ETA: 5s - loss: 0.6874 - accuracy: 0.54 - ETA: 4s - loss: 0.6879 - accuracy: 0.54 - ETA: 4s - loss: 0.6880 - accuracy: 0.54 - ETA: 4s - loss: 0.6877 - accuracy: 0.55 - ETA: 4s - loss: 0.6879 - accuracy: 0.54 - ETA: 4s - loss: 0.6882 - accuracy: 0.54 - ETA: 4s - loss: 0.6882 - accuracy: 0.54 - ETA: 4s - loss: 0.6883 - accuracy: 0.54 - ETA: 4s - loss: 0.6883 - accuracy: 0.54 - ETA: 4s - loss: 0.6883 - accu

40268/40268 [==============================] - ETA: 5s - loss: 0.6894 - accuracy: 0.55 - ETA: 6s - loss: 0.6888 - accuracy: 0.53 - ETA: 6s - loss: 0.6876 - accuracy: 0.54 - ETA: 6s - loss: 0.6861 - accuracy: 0.55 - ETA: 6s - loss: 0.6888 - accuracy: 0.53 - ETA: 6s - loss: 0.6897 - accuracy: 0.53 - ETA: 6s - loss: 0.6895 - accuracy: 0.53 - ETA: 6s - loss: 0.6896 - accuracy: 0.53 - ETA: 6s - loss: 0.6899 - accuracy: 0.53 - ETA: 5s - loss: 0.6899 - accuracy: 0.53 - ETA: 5s - loss: 0.6900 - accuracy: 0.53 - ETA: 5s - loss: 0.6900 - accuracy: 0.53 - ETA: 5s - loss: 0.6901 - accuracy: 0.53 - ETA: 5s - loss: 0.6894 - accuracy: 0.53 - ETA: 5s - loss: 0.6897 - accuracy: 0.53 - ETA: 5s - loss: 0.6900 - accuracy: 0.53 - ETA: 5s - loss: 0.6898 - accuracy: 0.53 - ETA: 5s - loss: 0.6896 - accuracy: 0.53 - ETA: 5s - loss: 0.6895 - accuracy: 0.53 - ETA: 5s - loss: 0.6898 - accuracy: 0.53 - ETA: 5s - loss: 0.6897 - accuracy: 0.53 - ETA: 5s - loss: 0.6894 - accuracy: 0.53 - ETA: 5s - loss: 0.6895 - accu

40268/40268 [==============================] - ETA: 5s - loss: 0.6861 - accuracy: 0.55 - ETA: 6s - loss: 0.6904 - accuracy: 0.52 - ETA: 6s - loss: 0.6902 - accuracy: 0.51 - ETA: 6s - loss: 0.6904 - accuracy: 0.51 - ETA: 6s - loss: 0.6897 - accuracy: 0.52 - ETA: 6s - loss: 0.6901 - accuracy: 0.52 - ETA: 6s - loss: 0.6903 - accuracy: 0.52 - ETA: 6s - loss: 0.6903 - accuracy: 0.52 - ETA: 6s - loss: 0.6896 - accuracy: 0.53 - ETA: 5s - loss: 0.6890 - accuracy: 0.53 - ETA: 5s - loss: 0.6882 - accuracy: 0.53 - ETA: 5s - loss: 0.6879 - accuracy: 0.54 - ETA: 5s - loss: 0.6884 - accuracy: 0.53 - ETA: 5s - loss: 0.6882 - accuracy: 0.53 - ETA: 5s - loss: 0.6886 - accuracy: 0.53 - ETA: 5s - loss: 0.6880 - accuracy: 0.53 - ETA: 5s - loss: 0.6877 - accuracy: 0.54 - ETA: 5s - loss: 0.6877 - accuracy: 0.54 - ETA: 5s - loss: 0.6873 - accuracy: 0.54 - ETA: 5s - loss: 0.6871 - accuracy: 0.54 - ETA: 5s - loss: 0.6874 - accuracy: 0.54 - ETA: 5s - loss: 0.6874 - accuracy: 0.54 - ETA: 5s - loss: 0.6873 - accu

...

40268/40268 [==============================] - ETA: 5s - loss: 0.4622 - accuracy: 0.76 - ETA: 6s - loss: 0.5299 - accuracy: 0.69 - ETA: 6s - loss: 0.5364 - accuracy: 0.69 - ETA: 6s - loss: 0.5441 - accuracy: 0.68 - ETA: 5s - loss: 0.5399 - accuracy: 0.68 - ETA: 5s - loss: 0.5371 - accuracy: 0.69 - ETA: 5s - loss: 0.5375 - accuracy: 0.69 - ETA: 5s - loss: 0.5323 - accuracy: 0.70 - ETA: 5s - loss: 0.5304 - accuracy: 0.70 - ETA: 5s - loss: 0.5335 - accuracy: 0.70 - ETA: 5s - loss: 0.5312 - accuracy: 0.70 - ETA: 5s - loss: 0.5332 - accuracy: 0.70 - ETA: 5s - loss: 0.5315 - accuracy: 0.70 - ETA: 5s - loss: 0.5315 - accuracy: 0.70 - ETA: 5s - loss: 0.5317 - accuracy: 0.70 - ETA: 5s - loss: 0.5329 - accuracy: 0.70 - ETA: 5s - loss: 0.5323 - accuracy: 0.70 - ETA: 5s - loss: 0.5316 - accuracy: 0.70 - ETA: 5s - loss: 0.5331 - accuracy: 0.70 - ETA: 5s - loss: 0.5331 - accuracy: 0.70 - ETA: 5s - loss: 0.5315 - accuracy: 0.70 - ETA: 4s - loss: 0.5335 - accuracy: 0.70 - ETA: 4s - loss: 0.5345 - accu

40268/40268 [==============================] - ETA: 5s - loss: 0.6004 - accuracy: 0.60 - ETA: 6s - loss: 0.5136 - accuracy: 0.70 - ETA: 6s - loss: 0.5338 - accuracy: 0.70 - ETA: 5s - loss: 0.5349 - accuracy: 0.70 - ETA: 5s - loss: 0.5353 - accuracy: 0.70 - ETA: 5s - loss: 0.5325 - accuracy: 0.70 - ETA: 5s - loss: 0.5438 - accuracy: 0.69 - ETA: 5s - loss: 0.5438 - accuracy: 0.69 - ETA: 5s - loss: 0.5414 - accuracy: 0.70 - ETA: 5s - loss: 0.5435 - accuracy: 0.69 - ETA: 5s - loss: 0.5378 - accuracy: 0.70 - ETA: 6s - loss: 0.5382 - accuracy: 0.70 - ETA: 6s - loss: 0.5419 - accuracy: 0.70 - ETA: 5s - loss: 0.5412 - accuracy: 0.70 - ETA: 5s - loss: 0.5399 - accuracy: 0.70 - ETA: 5s - loss: 0.5411 - accuracy: 0.70 - ETA: 5s - loss: 0.5400 - accuracy: 0.70 - ETA: 5s - loss: 0.5385 - accuracy: 0.70 - ETA: 5s - loss: 0.5407 - accuracy: 0.70 - ETA: 5s - loss: 0.5396 - accuracy: 0.70 - ETA: 5s - loss: 0.5400 - accuracy: 0.70 - ETA: 5s - loss: 0.5409 - accuracy: 0.70 - ETA: 5s - loss: 0.5413 - accu

40268/40268 [==============================] - ETA: 6s - loss: 0.5479 - accuracy: 0.64 - ETA: 6s - loss: 0.5822 - accuracy: 0.65 - ETA: 5s - loss: 0.5619 - accuracy: 0.67 - ETA: 5s - loss: 0.5501 - accuracy: 0.68 - ETA: 5s - loss: 0.5385 - accuracy: 0.70 - ETA: 5s - loss: 0.5381 - accuracy: 0.70 - ETA: 5s - loss: 0.5323 - accuracy: 0.70 - ETA: 5s - loss: 0.5317 - accuracy: 0.70 - ETA: 5s - loss: 0.5343 - accuracy: 0.70 - ETA: 5s - loss: 0.5327 - accuracy: 0.70 - ETA: 5s - loss: 0.5321 - accuracy: 0.71 - ETA: 5s - loss: 0.5309 - accuracy: 0.71 - ETA: 5s - loss: 0.5304 - accuracy: 0.71 - ETA: 5s - loss: 0.5299 - accuracy: 0.71 - ETA: 5s - loss: 0.5303 - accuracy: 0.71 - ETA: 5s - loss: 0.5321 - accuracy: 0.70 - ETA: 5s - loss: 0.5308 - accuracy: 0.71 - ETA: 5s - loss: 0.5311 - accuracy: 0.71 - ETA: 5s - loss: 0.5333 - accuracy: 0.70 - ETA: 5s - loss: 0.5342 - accuracy: 0.70 - ETA: 5s - loss: 0.5344 - accuracy: 0.70 - ETA: 4s - loss: 0.5354 - accuracy: 0.70 - ETA: 4s - loss: 0.5367 - accu

40268/40268 [==============================] - ETA: 5s - loss: 0.5638 - accuracy: 0.68 - ETA: 5s - loss: 0.5193 - accuracy: 0.70 - ETA: 5s - loss: 0.5169 - accuracy: 0.71 - ETA: 5s - loss: 0.5128 - accuracy: 0.72 - ETA: 5s - loss: 0.5130 - accuracy: 0.71 - ETA: 5s - loss: 0.5097 - accuracy: 0.72 - ETA: 5s - loss: 0.5153 - accuracy: 0.71 - ETA: 5s - loss: 0.5189 - accuracy: 0.71 - ETA: 5s - loss: 0.5209 - accuracy: 0.71 - ETA: 5s - loss: 0.5180 - accuracy: 0.71 - ETA: 5s - loss: 0.5218 - accuracy: 0.71 - ETA: 5s - loss: 0.5224 - accuracy: 0.71 - ETA: 5s - loss: 0.5275 - accuracy: 0.70 - ETA: 5s - loss: 0.5292 - accuracy: 0.70 - ETA: 5s - loss: 0.5305 - accuracy: 0.70 - ETA: 5s - loss: 0.5291 - accuracy: 0.70 - ETA: 5s - loss: 0.5271 - accuracy: 0.70 - ETA: 5s - loss: 0.5262 - accuracy: 0.70 - ETA: 5s - loss: 0.5276 - accuracy: 0.70 - ETA: 5s - loss: 0.5269 - accuracy: 0.70 - ETA: 5s - loss: 0.5267 - accuracy: 0.70 - ETA: 5s - loss: 0.5274 - accuracy: 0.70 - ETA: 5s - loss: 0.5285 - accu

40268/40268 [==============================] - ETA: 5s - loss: 0.4937 - accuracy: 0.76 - ETA: 6s - loss: 0.5212 - accuracy: 0.72 - ETA: 5s - loss: 0.5257 - accuracy: 0.71 - ETA: 5s - loss: 0.5315 - accuracy: 0.71 - ETA: 5s - loss: 0.5275 - accuracy: 0.71 - ETA: 5s - loss: 0.5163 - accuracy: 0.72 - ETA: 5s - loss: 0.5177 - accuracy: 0.72 - ETA: 5s - loss: 0.5212 - accuracy: 0.71 - ETA: 5s - loss: 0.5223 - accuracy: 0.72 - ETA: 5s - loss: 0.5196 - accuracy: 0.72 - ETA: 5s - loss: 0.5200 - accuracy: 0.72 - ETA: 5s - loss: 0.5194 - accuracy: 0.71 - ETA: 5s - loss: 0.5233 - accuracy: 0.71 - ETA: 5s - loss: 0.5220 - accuracy: 0.71 - ETA: 5s - loss: 0.5240 - accuracy: 0.71 - ETA: 5s - loss: 0.5234 - accuracy: 0.71 - ETA: 5s - loss: 0.5235 - accuracy: 0.71 - ETA: 5s - loss: 0.5237 - accuracy: 0.71 - ETA: 5s - loss: 0.5223 - accuracy: 0.71 - ETA: 5s - loss: 0.5234 - accuracy: 0.71 - ETA: 5s - loss: 0.5228 - accuracy: 0.71 - ETA: 4s - loss: 0.5222 - accuracy: 0.71 - ETA: 4s - loss: 0.5212 - accu

In [24]:
# evaluate the model
loss, accuracy = model.evaluate(reshape_x_test, y_test, verbose=0)
print('Accuracy: %f' % (accuracy))

Accuracy: 0.498836


In [25]:
%%javascript
Jupyter.notebook.save_checkpoint()

<IPython.core.display.Javascript object>

In [26]:
from time import sleep
sleep(60)

In [27]:
from shutil import copyfile
copyfile('RNN_Model_Base.ipynb', folder + '/RNN_Model_Base.ipynb' )

'Resultados/Embed_promedio/RNN_Model_Base.ipynb'

### Embedding Suma

In [28]:
# # Selecciono el archivo con el que se corre el modelo
# data = embeddings_sum_individual

In [29]:
# # Se separa en train y test
# x_train = data.drop([data.columns[0],"Top","Label", "Date"], axis=1)[:num_training]
# x_test = data.drop([data.columns[0],"Top",'Label', 'Date'], axis=1)[num_training:]
# y_train = data["Label"].values[:num_training]
# y_test = data["Label"].values[num_training:]

In [30]:
# x_train_array = x_train.to_numpy()
# reshape_x_train = x_train_array.reshape(40268, 1, 300)
# reshape_x_train.shape

In [31]:
# x_test_array = x_test.to_numpy()
# reshape_x_test = x_test_array.reshape(9450, 1, 300)
# reshape_x_test.shape

In [32]:
# from numpy.testing import assert_allclose
# from keras.models import Sequential, load_model
# from keras.layers import LSTM, Dropout, Dense
# from keras.callbacks import ModelCheckpoint

In [33]:
# n_units = 10

# model = Sequential()
# model.add(LSTM(n_units, input_shape=(1,300), return_sequences=True))
# model.add(Dropout(0.2))
# model.add(LSTM(n_units, return_sequences=False))
# model.add(Dense(1, activation='sigmoid'))
# #model.compile(loss='mean_squared_error', optimizer='adam')
# # compile the model
# model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
# # define the checkpoint
# filepath="word2vec-{epoch:02d}-{loss:.4f}.hdf5"
# checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
# callbacks_list = [checkpoint]

In [34]:
# # fit the model
# model.fit(reshape_x_train, y_train, epochs=50, batch_size=500, callbacks=callbacks_list)
# model.summary()

In [35]:
# # evaluate the model
# loss, accuracy = model.evaluate(reshape_x_test, y_test, verbose=0)
# print('Accuracy: %f' % (accuracy))